In [1]:
from collections import defaultdict

In [52]:
import json

In [152]:
def get_text():
    with open('../fb-data-analysis/xroads.txt', 'r') as f:
#     with open('../mc-shortener/eric.txt', 'r') as f:
        return f.read()

In [153]:
def from_text(text, n):
    """Create a probability trie structure from this text."""
    frequency_trie = _make_frequency_trie(text, n)
    probability_trie = _from_frequency_trie(frequency_trie)
    return probability_trie


def _make_ngrams(text, n):
    """Return all n-grams of a specified length in a text."""
    for i in range(len(text) - n):
        yield text[i:i + n]


def _make_empty_trie(n, end_type=int):
    """Return a data structure such that after n-1 associative array
    deferences, you get a defaultdict of type end_type"""
    if n == 1:
        return defaultdict(end_type)
    if n > 1:
        return defaultdict(partial(_make_empty_trie, n - 1, end_type))
    raise ValueError


def _update_trie(trie, key, func):
    """Apply func to the value of trie at key."""
    # if we have a defaultdict of a plain type
    if len(key) == 1:
        trie[key] = func(trie[key])
        return
    if len(key) > 1:
        _update_trie(trie[key[0]], key[1:], func)
        return
    raise ValueError


def _make_frequency_trie(text, n):
    trie = _make_empty_trie(n, end_type=int)
    for n_gram in _make_ngrams(text, n):
        _update_trie(trie, n_gram, lambda i: i + 1)
    return trie

In [154]:
def _as_dict(trie):
    """Convert from nested defaultdict to nested dict."""
    if not isinstance(trie, defaultdict):
        return trie
    return {k: _as_dict(v) for k, v in trie.items()}

In [155]:
trie = _make_frequency_trie(get_text(), 9)

In [156]:
def _compress_trie(trie):
    if not isinstance(trie, dict):
        return trie
    
    compressed_trie = dict()
    for key, value in trie.items():
        value = _compress_trie(value)
        if not isinstance(value, dict):
            compressed_trie[key] = value
            continue
        if len(value) == 1:
            subkey, subvalue = tuple(value.items())[0]
            compressed_trie[key + subkey] = subvalue
        else:
            compressed_trie[key] = value
    return compressed_trie

In [157]:
compressed_trie = _compress_trie(trie)

In [158]:
with open('/tmp/dump.json', 'w') as f:
    json.dump(compressed_trie, f)